# 4.9 Task Part 1

In [1]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
import scipy

## Question 3

In [2]:
customer_data = pd.read_csv (r'/Users/aaronreese/Desktop/Instacart Basket Analysis/2. Data/Original Data/customers.csv')

In [3]:
customer_data.head()

,user_id,First Name,Surnam,Gender,STATE,Age,date_joined,n_dependants,fam_status,income
0,26711,Deborah,Esquivel,Female,Missouri,48,1/1/2017,3,married,165665
1,33890,Patricia,Hart,Female,New Mexico,36,1/1/2017,0,single,59285
2,65803,Kenneth,Farley,Male,Idaho,35,1/1/2017,2,married,99568
3,125935,Michelle,Hicks,Female,Iowa,40,1/1/2017,0,single,42049
4,130797,Ann,Gilmore,Female,Maryland,26,1/1/2017,1,married,40374


## Question 4: Consistent Logic

In [17]:
customer_data.rename (columns={'First Name':'first_name','Surnam':'last_name','Gender':'gender','STATE':'state',
                              'Age':'age','n_dependants':'number_of_dependants'},inplace = True)

In [19]:
customer_data.drop(['fam_status'], axis = 1, inplace = True)

In [20]:
customer_data.head()

,user_id,first_name,last_name,gender,state,age,date_joined,number_of_dependants,income
0,26711,Deborah,Esquivel,Female,Missouri,48,1/1/2017,3,165665
1,33890,Patricia,Hart,Female,New Mexico,36,1/1/2017,0,59285
2,65803,Kenneth,Farley,Male,Idaho,35,1/1/2017,2,99568
3,125935,Michelle,Hicks,Female,Iowa,40,1/1/2017,0,42049
4,130797,Ann,Gilmore,Female,Maryland,26,1/1/2017,1,40374


## Question 5: Consistency Checks

In [21]:
customer_data.describe

<bound method NDFrame.describe of         user_id first_name last_name  gender           state  age date_joined  \
0         26711    Deborah  Esquivel  Female        Missouri   48    1/1/2017   
1         33890   Patricia      Hart  Female      New Mexico   36    1/1/2017   
2         65803    Kenneth    Farley    Male           Idaho   35    1/1/2017   
3        125935   Michelle     Hicks  Female            Iowa   40    1/1/2017   
4        130797        Ann   Gilmore  Female        Maryland   26    1/1/2017   
...         ...        ...       ...     ...             ...  ...         ...   
206204   168073       Lisa      Case  Female  North Carolina   44    4/1/2020   
206205    49635     Jeremy   Robbins    Male          Hawaii   62    4/1/2020   
206206   135902      Doris  Richmond  Female        Missouri   66    4/1/2020   
206207    81095       Rose   Rollins  Female      California   27    4/1/2020   
206208    80148    Cynthia     Noble  Female        New York   55    4/1/20

### Check for mixed types

In [27]:
df_test = pd.DataFrame()

In [28]:
df_test['mix'] = ['a','b',1,True]

In [29]:
for col in df_test.columns.tolist():
  weird = (df_test[[col]].applymap(type) != df_test[[col]].iloc[0].apply(type)).any(axis = 1)
  if len (df_test[weird]) > 0:
    print (col)

mix


### Missing Values

In [26]:
customer_data.isnull().sum()

user_id                     0
first_name              11259
last_name                   0
gender                      0
state                       0
age                         0
date_joined                 0
number_of_dependants        0
income                      0
dtype: int64

In [30]:
df_nan_first_name = customer_data[customer_data['first_name'].isnull() == True]

In [31]:
df_nan_first_name

,user_id,first_name,last_name,gender,state,age,date_joined,number_of_dependants,income
53,76659,NaN,Gilbert,Male,Colorado,26,1/1/2017,2,41709
73,13738,NaN,Frost,Female,Louisiana,39,1/1/2017,0,82518
82,89996,NaN,Dawson,Female,Oregon,52,1/1/2017,3,117099
99,96166,NaN,Oconnor,Male,Oklahoma,51,1/1/2017,1,155673
105,29778,NaN,Dawson,Female,Utah,63,1/1/2017,3,151819
...,...,...,...,...,...,...,...,...,...
206038,121317,NaN,Melton,Male,Pennsylvania,28,3/31/2020,3,87783
206044,200799,NaN,Copeland,Female,Hawaii,52,4/1/2020,2,108488
206090,167394,NaN,Frost,Female,Hawaii,61,4/1/2020,1,45275
206162,187532,NaN,Floyd,Female,California,39,4/1/2020,0,56325


### Duplicates

In [32]:
df_dups = customer_data[customer_data.duplicated()]

In [33]:
df_dups

,user_id,first_name,last_name,gender,state,age,date_joined,number_of_dependants,income


## Question 6: Combine Dataframes

In [34]:
ords_prods_merge = pd.read_pickle (r'/Users/aaronreese/Desktop/Instacart Basket Analysis/2. Data/Prepared Data/4-8_ords_prods_merge.pkl')

In [36]:
ords_prods_merge.head()

,order_id,user_id,eval_set,order_number,orders_day_of_week,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,...,price_range_loc,busiest_day,busiest_days,busiest_period_of_day,max_order,loyalty_flag,spender,spending_flag,order_frequency,frequency_flag
0,2539329,1,prior,1,2,8,NaN,196,1,0,...,Mid-range product,Regularly busy,Regularly busy,Fewest orders,10,New customer,14.0,High spender,30.0,Non-frequent customer
1,2398795,1,prior,2,3,7,15.0,196,1,1,...,Mid-range product,Regularly busy,Regularly busy,Fewest orders,10,New customer,14.0,High spender,30.0,Non-frequent customer
2,473747,1,prior,3,3,12,21.0,196,1,1,...,Mid-range product,Regularly busy,Regularly busy,Fewest orders,10,New customer,14.0,High spender,30.0,Non-frequent customer
3,2254736,1,prior,4,4,7,29.0,196,1,1,...,Mid-range product,Least busy,Slowest days,Fewest orders,10,New customer,14.0,High spender,30.0,Non-frequent customer
4,431534,1,prior,5,4,15,28.0,196,1,1,...,Mid-range product,Least busy,Slowest days,Fewest orders,10,New customer,14.0,High spender,30.0,Non-frequent customer


In [42]:
cust_ords_prods = ords_prods_merge.merge(customer_data, on = 'user_id', indicator = 'exists')

In [44]:
cust_ords_prods.head()

,order_id,user_id,eval_set,order_number,orders_day_of_week,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,...,frequency_flag,first_name,last_name,gender,state,age,date_joined,number_of_dependants,income,exists
0,2539329,1,prior,1,2,8,NaN,196,1,0,...,Non-frequent customer,Linda,Nguyen,Female,Alabama,31,2/17/2019,3,40423,both
1,2398795,1,prior,2,3,7,15.0,196,1,1,...,Non-frequent customer,Linda,Nguyen,Female,Alabama,31,2/17/2019,3,40423,both
2,473747,1,prior,3,3,12,21.0,196,1,1,...,Non-frequent customer,Linda,Nguyen,Female,Alabama,31,2/17/2019,3,40423,both
3,2254736,1,prior,4,4,7,29.0,196,1,1,...,Non-frequent customer,Linda,Nguyen,Female,Alabama,31,2/17/2019,3,40423,both
4,431534,1,prior,5,4,15,28.0,196,1,1,...,Non-frequent customer,Linda,Nguyen,Female,Alabama,31,2/17/2019,3,40423,both


In [46]:
cust_ords_prods['exists'].value_counts()

both          32434212
left_only            0
right_only           0
Name: exists, dtype: int64

### Export

In [47]:
cust_ords_prods.to_pickle('/Users/aaronreese/Desktop/Instacart Basket Analysis/2. Data/Prepared Data/cust_ords_prods.pkl')